In [1]:
import pandas as pd
import experiment_util as util
import models
import numpy as np
import random

np.random.seed(0)
random.seed(0)

In [2]:
raw_df=pd.read_csv("../Datasets/Kaggle/College/data.csv")

df=raw_df.copy()
#df = df.loc[~df.index.duplicated(keep='first')]
df

,type_school,school_accreditation,gender,interest,residence,parent_age,parent_salary,house_area,average_grades,parent_was_in_college,will_go_to_college
0,Academic,A,Male,Less Interested,Urban,56,6950000,83.0,84.09,False,True
1,Academic,A,Male,Less Interested,Urban,57,4410000,76.8,86.91,False,True
2,Academic,B,Female,Very Interested,Urban,50,6500000,80.6,87.43,False,True
3,Vocational,B,Male,Very Interested,Rural,49,6600000,78.2,82.12,True,True
4,Academic,A,Female,Very Interested,Urban,57,5250000,75.1,86.79,False,False
...,...,...,...,...,...,...,...,...,...,...,...
995,Vocational,A,Female,Very Interested,Rural,49,7420000,63.6,85.99,True,True
996,Academic,B,Female,Less Interested,Rural,51,7480000,84.3,89.72,True,True
997,Vocational,A,Male,Less Interested,Urban,49,5550000,75.2,79.56,False,True
998,Academic,B,Male,Uncertain,Rural,53,5840000,105.8,87.18,True,True


In [4]:
folds_index=np.array_split(df.sample(frac=1).index,5)
eval=util.evaluation(df,"gender","will_go_to_college","Male",folds_index)
pre_df=util.preporcess_transform(df)
result=pd.DataFrame([],columns=util.column)
result.columns.name="College"
result

College,Accuracy,False Omissin Rate Disparity,False Discover Rate Disparity,Predicted Positive Ratio Disparity,Predicted Prevalance Disparity


In [5]:
result=util.append_Series(result,eval.cross_validation("Random Forest",models.RandomForest,pre_df))
result

c:\Users\15783\source\repos\Ethical-Framework\Aequitas\experiment_util.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df=pd.concat([dataframe,pd.DataFrame([series.to_list()],index=[series.name],columns=column)])


College,Accuracy,False Omissin Rate Disparity,False Discover Rate Disparity,Predicted Positive Ratio Disparity,Predicted Prevalance Disparity
Random Forest,0.888,0.930915,0.998136,0.991813,1.050639


In [6]:
result=util.append_Series(result,eval.cross_validation("Gradient Boost",models.GradientBoost,pre_df))
result

College,Accuracy,False Omissin Rate Disparity,False Discover Rate Disparity,Predicted Positive Ratio Disparity,Predicted Prevalance Disparity
Random Forest,0.888,0.930915,0.998136,0.991813,1.050639
Gradient Boost,0.894,0.850596,0.898333,0.990580,1.053905


In [ ]:
result=util.append_Series(result,eval.cross_validation("Gradient Boost",models.Fairgbm,pre_df))
result

In [ ]:
result=util.append_Series(result,eval.cross_validation("Gradient Boost",models.Fairlearn,pre_df))
result

In [ ]:
result.to_csv("College_result.csv")

In [43]:
import importlib
importlib.reload(models)
importlib.reload(util)

<module 'experiment_util' from 'c:\\Users\\15783\\source\\repos\\Ethical-Framework\\Aequitas\\experiment_util.py'>